<section style="direction: ltr; text-align: center; margin: 30px;">

# **SQL-Fundamentals-Farsi**

</section>

<section style="direction: ltr; text-align: justyfy; margin: 30px;">

This project offers a versatile and automated solution for generating database reports in structured formats. By leveraging popular Python libraries such as Pandas, SQLAlchemy, and Openpyxl, it simplifies the process of extracting, displaying, and exporting data from SQL Server databases. With the added ability to handle both Excel and CSV outputs, it provides a flexible tool for data analysis and reporting needs.

</section>

---

### **Calling the SQL Server Connection Function**

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_connection(server, database, username=None, password=None):
    """
    Create a connection to the SQL Server database using SQLAlchemy.
    
    Parameters:
    - server: str, the name of the SQL Server
    - database: str, the name of the database
    - username: str, optional, SQL Server username (for SQL authentication)
    - password: str, optional, SQL Server password (for SQL authentication)
    
    Returns:
    - engine: sqlalchemy.engine.base.Engine, the SQLAlchemy engine object or None if the connection fails
    """
    try:
        if username and password:
            connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
        else:
            connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
        
        engine = create_engine(connection_string)
        
        # Test connection
        with engine.connect() as conn:
            logging.info("Connection successful!")
        
        return engine

    except SQLAlchemyError as e:
        logging.error(f"Connection failed: {e}")
        return None

***

### **Check if the database connection is successful**

In [2]:
def check_connection(engine):
    if engine:
        logging.info("Database connection established successfully.")
        logging.info(f"Server: {server}")
        logging.info(f"Database: {database}")
        return True
    else:
        logging.error("Failed to establish database connection.")
        return False

# Call the create_connection function
server = r'imsalione-pc\imsalionedb'
database = 'AdventureWorksLT2022'

engine = create_connection(server, database)

# Use the check_connection function to verify the connection
if check_connection(engine):
    # Proceed with database operations
    pass
else:
    # Handle the failed connection
    pass

2024-10-10 13:21:11,296 - INFO - Connection successful!
2024-10-10 13:21:11,297 - INFO - Database connection established successfully.
2024-10-10 13:21:11,299 - INFO - Server: imsalione-pc\imsalionedb
2024-10-10 13:21:11,300 - INFO - Database: AdventureWorksLT2022


***

### **Fetching and Displaying Data from SQL Server**

In [3]:
import pandas as pd
from sqlalchemy import exc
from pathlib import Path
from prettytable import PrettyTable

# Define the directory where you want to save the report files
output_directory = Path("C:/Reports")  # Adjust the path as needed
output_directory.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists

def truncate(value, width):
    return str(value)[:width] + "..." if len(str(value)) > width else str(value)

def fetch_and_display_queries(engine, queries, pnum=10, export_to_excel=False, export_to_csv=False):
    """
    Fetch data from multiple SQL queries and optionally export to Excel or CSV.
    
    Parameters:
    - engine: SQLAlchemy engine object.
    - queries: List of SQL query strings.
    - pnum: int, number of rows to display (default is 10).
    - export_to_excel: bool, whether to export data to an Excel file (default is False).
    - export_to_csv: bool, whether to export data to a CSV file (default is False).
    """
    
    if engine:
        # If exporting to Excel, create the writer object for the single file
        if export_to_excel:
            file_path = output_directory / f"report_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.xlsx"
            writer = pd.ExcelWriter(file_path, engine='openpyxl')

        for i, query in enumerate(queries, 1):

            print(f"Executing query {i}:\n{query}")
            try:
                # Load data into a pandas DataFrame
                df = pd.read_sql(query, engine)
                
                # Display the first pnum rows (or default 10 rows if pnum is not provided)
                table = PrettyTable()
                table.field_names = df.columns
                
                # Limit the width of each column to 20 characters
                for row in df.head(pnum).itertuples(index=False):
                    table.add_row([truncate(value, 25) for value in row])
                
                print(table)
                
                # Print the number of records in the query result
                record_count = len(df)
                print(f"Query {i} returned {record_count} records.\n")
                
                # Export to Excel in different sheets if requested
                if export_to_excel:
                    sheet_name = f"Query_{i}"
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"Data from query {i} exported to sheet {sheet_name} in {file_path}")
                
                # Export to CSV if requested (each query in a different CSV file)
                if export_to_csv:
                    csv_file_path = output_directory / f"report_{i}_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.csv"
                    df.to_csv(csv_file_path, index=False)
                    print(f"Data from query {i} exported to CSV at {csv_file_path}")

            except exc.SQLAlchemyError as db_error:
                print(f"Database error occurred during query {i}: {db_error}")
            except Exception as e:
                print(f"An error occurred during query {i}: {e}")

        # Save the Excel file with all sheets after the loop finishes
        if export_to_excel:
            writer.close()  # Use close() instead of save()
            print(f"All data exported to {file_path}")

    else:
        print("No database connection.")

# Example usage:
# queries = ["""SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Customer';""", ...]
# fetch_and_display_multiple_queries(engine, queries, pnum=20, export_to_excel=True)

***

### **Executing SQL Query to Fetch and Display Table Information**

In [4]:
queries = ["""SELECT TABLE_SCHEMA, TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES"""]

fetch_and_display_queries(engine, queries, pnum=20)

Executing query 1:
SELECT TABLE_SCHEMA, TABLE_NAME 
FROM INFORMATION_SCHEMA.TABLES
+--------------+------------------------------+
| TABLE_SCHEMA |          TABLE_NAME          |
+--------------+------------------------------+
|     dbo      |           ErrorLog           |
|     dbo      |         BuildVersion         |
|   SalesLT    |           Address            |
|   SalesLT    |           Customer           |
|   SalesLT    |       CustomerAddress        |
|   SalesLT    |           Product            |
|   SalesLT    |       ProductCategory        |
|   SalesLT    |      ProductDescription      |
|   SalesLT    |         ProductModel         |
|   SalesLT    | ProductModelProductDescri... |
|   SalesLT    |       SalesOrderDetail       |
|   SalesLT    |       SalesOrderHeader       |
|   SalesLT    |    vProductAndDescription    |
|   SalesLT    | vProductModelCatalogDescr... |
|   SalesLT    |      vGetAllCategories       |
+--------------+------------------------------+
Query

In [5]:
queries = ["""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Customer';""",

"""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';""",

"""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'SalesOrderDetail';""",

"""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'ProductModel';"""
]

fetch_and_display_queries(engine, queries, pnum=20)

Executing query 1:
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Customer';
+--------------+------------------+
| COLUMN_NAME  |    DATA_TYPE     |
+--------------+------------------+
|  CustomerID  |       int        |
|  NameStyle   |       bit        |
|    Title     |     nvarchar     |
|  FirstName   |     nvarchar     |
|  MiddleName  |     nvarchar     |
|   LastName   |     nvarchar     |
|    Suffix    |     nvarchar     |
| CompanyName  |     nvarchar     |
| SalesPerson  |     nvarchar     |
| EmailAddress |     nvarchar     |
|    Phone     |     nvarchar     |
| PasswordHash |     varchar      |
| PasswordSalt |     varchar      |
|   rowguid    | uniqueidentifier |
| ModifiedDate |     datetime     |
+--------------+------------------+
Query 1 returned 15 records.

Executing query 2:
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';
+------------------------+------------------+
|      COLUMN_NAM

In [6]:
query = ["""SELECT FirstName, LastName, 
FirstName + ' ' + LastName AS FullName 
FROM SalesLT.Customer
ORDER BY LastName DESC"""]

fetch_and_display_queries(engine, query, 10)

Executing query 1:
SELECT FirstName, LastName, 
FirstName + ' ' + LastName AS FullName 
FROM SalesLT.Customer
ORDER BY LastName DESC
+-----------+-----------+-------------------+
| FirstName |  LastName |      FullName     |
+-----------+-----------+-------------------+
|  Caroline |  Vicknair | Caroline Vicknair |
|  Caroline |  Vicknair | Caroline Vicknair |
|   Robert  |   Vessa   |    Robert Vessa   |
|   Robert  |   Vessa   |    Robert Vessa   |
|   Wanda   |   Vernon  |    Wanda Vernon   |
|   Wanda   |   Vernon  |    Wanda Vernon   |
|    Dora   |   Verdad  |    Dora Verdad    |
|    Dora   |   Verdad  |    Dora Verdad    |
|    Raja   | Venugopal |   Raja Venugopal  |
|    Raja   | Venugopal |   Raja Venugopal  |
+-----------+-----------+-------------------+
Query 1 returned 847 records.



In [7]:
query = ["""SELECT AVG(OrderQty) AS AverageQTY 
FROM SalesLT.SalesOrderDetail""", 

"""SELECT OrderQty, ProductID, UnitPrice, CASE
WHEN OrderQty < 3 THEN N'کم فروش'
WHEN OrderQty >= 3 THEN N'پرفروش'
END AS SalesClass 
FROM SalesLT.SalesOrderDetail"""]

fetch_and_display_queries(engine, query, 5)

Executing query 1:
SELECT AVG(OrderQty) AS AverageQTY 
FROM SalesLT.SalesOrderDetail
+------------+
| AverageQTY |
+------------+
|     3      |
+------------+
Query 1 returned 1 records.

Executing query 2:
SELECT OrderQty, ProductID, UnitPrice, CASE
WHEN OrderQty < 3 THEN N'کم فروش'
WHEN OrderQty >= 3 THEN N'پرفروش'
END AS SalesClass 
FROM SalesLT.SalesOrderDetail
+----------+-----------+-----------+------------+
| OrderQty | ProductID | UnitPrice | SalesClass |
+----------+-----------+-----------+------------+
|    1     |    836    |  356.898  |  کم فروش   |
|    1     |    822    |  356.898  |  کم فروش   |
|    1     |    907    |    63.9   |  کم فروش   |
|    4     |    905    |  218.454  |   پرفروش   |
|    2     |    983    |  461.694  |  کم فروش   |
+----------+-----------+-----------+------------+
Query 2 returned 542 records.



In [8]:
query = ["""SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';
"""]

fetch_and_display_queries(engine, query, pnum=17)

Executing query 1:
SELECT COLUMN_NAME, DATA_TYPE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Product';

+------------------------+------------------+
|      COLUMN_NAME       |    DATA_TYPE     |
+------------------------+------------------+
|       ProductID        |       int        |
|          Name          |     nvarchar     |
|     ProductNumber      |     nvarchar     |
|         Color          |     nvarchar     |
|      StandardCost      |      money       |
|       ListPrice        |      money       |
|          Size          |     nvarchar     |
|         Weight         |     decimal      |
|   ProductCategoryID    |       int        |
|     ProductModelID     |       int        |
|     SellStartDate      |     datetime     |
|      SellEndDate       |     datetime     |
|    DiscontinuedDate    |     datetime     |
|     ThumbNailPhoto     |    varbinary     |
| ThumbnailPhotoFileName |     nvarchar     |
|        rowguid         | uniqueidentifier |
|      Modifie

In [9]:
query = ["""SELECT ProductNumber, StandardCost, Size, Weight 
FROM SalesLT.Product"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT ProductNumber, StandardCost, Size, Weight 
FROM SalesLT.Product
+---------------+--------------+------+---------+
| ProductNumber | StandardCost | Size |  Weight |
+---------------+--------------+------+---------+
|   FR-R92B-58  |   1059.31    |  58  | 1016.04 |
|   FR-R92R-58  |   1059.31    |  58  | 1016.04 |
|   HL-U509-R   |   13.0863    | None |   nan   |
|    HL-U509    |   13.0863    | None |   nan   |
|   SO-B909-M   |    3.3963    |  M   |   nan   |
|   SO-B909-L   |    3.3963    |  L   |   nan   |
|   HL-U509-B   |   13.0863    | None |   nan   |
|    CA-1098    |    6.9223    | None |   nan   |
|   LJ-0192-S   |   38.4923    |  S   |   nan   |
|   LJ-0192-M   |   38.4923    |  M   |   nan   |
+---------------+--------------+------+---------+
Query 1 returned 295 records.



In [10]:
query = ["""SELECT AVG(YEAR(ModifiedDate)) AS DateAverage,
Max(YEAR(ModifiedDate)) AS MaxYear,
Min(YEAR(ModifiedDate)) AS MinYear
FROM SalesLT.ProductDescription"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT AVG(YEAR(ModifiedDate)) AS DateAverage,
Max(YEAR(ModifiedDate)) AS MaxYear,
Min(YEAR(ModifiedDate)) AS MinYear
FROM SalesLT.ProductDescription
+-------------+---------+---------+
| DateAverage | MaxYear | MinYear |
+-------------+---------+---------+
|     2007    |   2008  |   2007  |
+-------------+---------+---------+
Query 1 returned 1 records.



In [11]:
query = ["""SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
ORDER BY YEAR(ModifiedDate) DESC"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
ORDER BY YEAR(ModifiedDate) DESC
+-----------+-------------+-----------+
| FirstName |   LastName  | FirstSale |
+-----------+-------------+-----------+
|  Margaret |    Smith    |    2009   |
|  Frances  |    Adams    |    2009   |
|    Jay    |    Adams    |    2009   |
| Catherine |     Abel    |    2009   |
|    Kim    | Abercrombie |    2009   |
| Catherine |     Abel    |    2009   |
|    Kim    | Abercrombie |    2009   |
|  Frances  |    Adams    |    2009   |
|  Margaret |    Smith    |    2009   |
|    Jay    |    Adams    |    2009   |
+-----------+-------------+-----------+
Query 1 returned 847 records.



In [12]:
query = ["""SELECT CompanyName
         FROM SalesLT.Customer"""]

fetch_and_display_queries(engine, query, pnum=20)

Executing query 1:
SELECT CompanyName
         FROM SalesLT.Customer
+------------------------------+
|         CompanyName          |
+------------------------------+
|         A Bike Store         |
|      Progressive Sports      |
|   Advanced Bike Components   |
|    Modular Cycle Systems     |
| Metropolitan Sports Suppl... |
|   Aerobic Exercise Company   |
|       Associated Bikes       |
|     Rural Cycle Emporium     |
|         Sharp Bikes          |
|     Bikes and Motorbikes     |
|     Bulk Discount Store      |
|        Catalog Store         |
|      Center Cycle Shop       |
|    Central Discount Store    |
|    Chic Department Stores    |
|        Travel Systems        |
|          Bike World          |
|  Eastside Department Store   |
|    Coalition Bike Company    |
|    Commuter Bicycle Store    |
+------------------------------+
Query 1 returned 847 records.



In [13]:
query = ["""SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
WHERE CompanyName = 'Coalition Bike Company'"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
WHERE CompanyName = 'Coalition Bike Company'
+-----------+----------+-----------+
| FirstName | LastName | FirstSale |
+-----------+----------+-----------+
|   Donald  | Blanton  |    2006   |
|   Donald  | Blanton  |    2006   |
+-----------+----------+-----------+
Query 1 returned 2 records.



In [14]:
query = ["""SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
WHERE CompanyName IN ('Bike World', 'Commuter Bicycle Store')"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
WHERE CompanyName IN ('Bike World', 'Commuter Bicycle Store')
+-----------+-----------+-----------+
| FirstName |  LastName | FirstSale |
+-----------+-----------+-----------+
|   Kerim   |   Hanif   |    2006   |
|   Jackie  | Blackwell |    2007   |
|   Jackie  | Blackwell |    2007   |
|   Kerim   |   Hanif   |    2006   |
+-----------+-----------+-----------+
Query 1 returned 4 records.



In [15]:
query = ["""SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
WHERE YEAR(ModifiedDate) = 2006"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT FirstName, LastName, YEAR(ModifiedDate) AS FirstSale
FROM SalesLT.Customer
WHERE YEAR(ModifiedDate) = 2006
+-------------+------------+-----------+
|  FirstName  |  LastName  | FirstSale |
+-------------+------------+-----------+
|    Keith    |   Harris   |    2006   |
|    Janet    |   Gates    |    2006   |
|     Lucy    | Harrington |    2006   |
|   Dominic   |    Gash    |    2006   |
|   Kathleen  |   Garza    |    2006   |
|    Johnny   |   Caprio   |    2006   |
| Christopher |    Beck    |    2006   |
|   Jinghao   |    Liu     |    2006   |
|    Kerim    |   Hanif    |    2006   |
|    Kevin    |    Liu     |    2006   |
+-------------+------------+-----------+
Query 1 returned 313 records.



In [16]:
query = ["""SELECT DISTINCT(YEAR(ModifiedDate)) AS SaleYear,
DATEDIFF(MONTH, modifieddate, GETDATE()) AS SaleCount 
FROM SalesLT.SalesOrderDetail"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT DISTINCT(YEAR(ModifiedDate)) AS SaleYear,
DATEDIFF(MONTH, modifieddate, GETDATE()) AS SaleCount 
FROM SalesLT.SalesOrderDetail
+----------+-----------+
| SaleYear | SaleCount |
+----------+-----------+
|   2008   |    196    |
+----------+-----------+
Query 1 returned 1 records.



In [17]:
query = ["""SELECT CustomerID, FirstName, LastName , DATEDIFF(YEAR, ModifiedDate, GETDATE())
AS experience, 
CASE 
WHEN DATEDIFF(YEAR, ModifiedDate, GETDATE()) > 18 THEN 'G1'
WHEN DATEDIFF(YEAR, ModifiedDate, GETDATE()) <= 18 THEN 'G2'
END AS experienceClass
FROM SalesLT.Customer"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT CustomerID, FirstName, LastName , DATEDIFF(YEAR, ModifiedDate, GETDATE())
AS experience, 
CASE 
WHEN DATEDIFF(YEAR, ModifiedDate, GETDATE()) > 18 THEN 'G1'
WHEN DATEDIFF(YEAR, ModifiedDate, GETDATE()) <= 18 THEN 'G2'
END AS experienceClass
FROM SalesLT.Customer
+------------+-----------+------------+------------+-----------------+
| CustomerID | FirstName |  LastName  | experience | experienceClass |
+------------+-----------+------------+------------+-----------------+
|     1      |  Orlando  |    Gee     |     19     |        G1       |
|     2      |   Keith   |   Harris   |     18     |        G2       |
|     3      |   Donna   |  Carreras  |     19     |        G1       |
|     4      |   Janet   |   Gates    |     18     |        G2       |
|     5      |    Lucy   | Harrington |     18     |        G2       |
|     6      |  Rosmarie |  Carroll   |     17     |        G2       |
|     7      |  Dominic  |    Gash    |     18     |        G2       |
| 

In [18]:
query = ["""SELECT FirstName, LastName FROM SalesLT.Customer
WHERE FirstName like 'B%'"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT FirstName, LastName FROM SalesLT.Customer
WHERE FirstName like 'B%'
+-----------+-----------+
| FirstName |  LastName |
+-----------+-----------+
|   Bryan   |  Hamilton |
|  Barbara  |   German  |
|   Betty   |   Haines  |
|   Brian   |   Groth   |
|   Bonnie  |   Lepro   |
|   Brigid  | Cavendish |
|   Blaine  |  Dockter  |
|   Brenda  |    Diaz   |
|   Brian   |  Johnson  |
|   Barry   |  Johnson  |
+-----------+-----------+
Query 1 returned 46 records.



In [19]:
query = ["""SELECT ProductID, isnull(UnitPrice, 0)
AS NewPrice
FROM SalesLT.SalesOrderDetail
ORDER BY NewPrice"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT ProductID, isnull(UnitPrice, 0)
AS NewPrice
FROM SalesLT.SalesOrderDetail
ORDER BY NewPrice
+-----------+----------+
| ProductID | NewPrice |
+-----------+----------+
|    873    |  1.374   |
|    873    |  1.374   |
|    873    |  1.374   |
|    870    |  2.8942  |
|    870    |  2.8942  |
|    870    |  2.994   |
|    870    |  2.994   |
|    870    |  2.994   |
|    870    |  2.994   |
|    870    |  2.994   |
+-----------+----------+
Query 1 returned 542 records.



In [20]:
query = ["""SELECT TOP 3 UnitPrice, ProductID
         FROM SalesLT.SalesOrderDetail
         ORDER BY UnitPrice DESC"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT TOP 3 UnitPrice, ProductID
         FROM SalesLT.SalesOrderDetail
         ORDER BY UnitPrice DESC
+-----------+-----------+
| UnitPrice | ProductID |
+-----------+-----------+
|  1466.01  |    793    |
|  1466.01  |    794    |
|  1466.01  |    795    |
+-----------+-----------+
Query 1 returned 3 records.



In [21]:
query = ["""SELECT FirstName, LastName 
         FROM SalesLT.Customer
         ORDER BY LastName DESC"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT FirstName, LastName 
         FROM SalesLT.Customer
         ORDER BY LastName DESC
+-----------+-----------+
| FirstName |  LastName |
+-----------+-----------+
|  Caroline |  Vicknair |
|  Caroline |  Vicknair |
|   Robert  |   Vessa   |
|   Robert  |   Vessa   |
|   Wanda   |   Vernon  |
|   Wanda   |   Vernon  |
|    Dora   |   Verdad  |
|    Dora   |   Verdad  |
|    Raja   | Venugopal |
|    Raja   | Venugopal |
+-----------+-----------+
Query 1 returned 847 records.



In [22]:
query = ["""SELECT TOP 6 FirstName, LastName, CustomerID
         FROM SalesLT.Customer
         ORDER BY ModifiedDate DESC"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT TOP 6 FirstName, LastName, CustomerID
         FROM SalesLT.Customer
         ORDER BY ModifiedDate DESC
+-----------+----------+------------+
| FirstName | LastName | CustomerID |
+-----------+----------+------------+
|    Jay    |  Adams   |    544     |
|    Jay    |  Adams   |   29492    |
|  Margaret |  Smith   |    488     |
|  Margaret |  Smith   |   29490    |
|  Frances  |  Adams   |    491     |
|  Frances  |  Adams   |   29489    |
+-----------+----------+------------+
Query 1 returned 6 records.



In [23]:
query = ["""SELECT ProductID, UnitPrice
         FROM SalesLT.SalesOrderDetail 
         WHERE UnitPrice BETWEEN 7 AND 20"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT ProductID, UnitPrice
         FROM SalesLT.SalesOrderDetail 
         WHERE UnitPrice BETWEEN 7 AND 20
+-----------+-----------+
| ProductID | UnitPrice |
+-----------+-----------+
|    859    |   14.694  |
|    859    |   14.694  |
|    858    |   14.694  |
|    860    |   14.694  |
|    711    |  19.2445  |
|    858    |   14.694  |
|    859    |   14.694  |
|    860    |   14.694  |
|    859    |   14.694  |
|    858    |   14.694  |
+-----------+-----------+
Query 1 returned 20 records.



In [24]:
query = ["""select ProductModelID AS ModelID, Name, YEAR(ModifiedDate) AS Year
         FROM SalesLT.ProductModel
         WHERE YEAR(ModifiedDate) < 2006
         """]

fetch_and_display_queries(engine, query)

Executing query 1:
select ProductModelID AS ModelID, Name, YEAR(ModifiedDate) AS Year
         FROM SalesLT.ProductModel
         WHERE YEAR(ModifiedDate) < 2006
         
+---------+-------------------------+------+
| ModelID |           Name          | Year |
+---------+-------------------------+------+
|    2    |       Cycling Cap       | 2005 |
|    5    |    HL Mountain Frame    | 2005 |
|    6    |      HL Road Frame      | 2002 |
|    9    |      LL Road Frame      | 2005 |
|    11   | Long-Sleeve Logo Jersey | 2005 |
|    16   |      ML Road Frame      | 2005 |
|    18   |   Mountain Bike Socks   | 2005 |
|    19   |       Mountain-100      | 2005 |
|    25   |         Road-150        | 2005 |
|    28   |         Road-450        | 2005 |
+---------+-------------------------+------+
Query 1 returned 13 records.



In [25]:
query = ["""SELECT
AVG(UnitPrice) as AvgPrice,
Max(UnitPrice) as MaxPrice,
Min(UnitPrice) as MinPrice
FROM SalesLT.SalesOrderDetail"""]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT
AVG(UnitPrice) as AvgPrice,
Max(UnitPrice) as MaxPrice,
Min(UnitPrice) as MinPrice
FROM SalesLT.SalesOrderDetail
+----------+----------+----------+
| AvgPrice | MaxPrice | MinPrice |
+----------+----------+----------+
|  382.87  | 1466.01  |  1.374   |
+----------+----------+----------+
Query 1 returned 1 records.



In [26]:
query = ["SELECT COUNT(*) AS NumberOfProducts FROM SalesLT.Product"]

fetch_and_display_queries(engine, query)

Executing query 1:
SELECT COUNT(*) AS NumberOfProducts FROM SalesLT.Product
+------------------+
| NumberOfProducts |
+------------------+
|       295        |
+------------------+
Query 1 returned 1 records.



In [39]:
query = ["""select count(*) AS CountOfShopping,
year(ModifiedDate) as FirstShopping
from SalesLT.Customer
group by year(ModifiedDate)
order by CountOfShopping desc"""]

fetch_and_display_queries(engine, query)

Executing query 1:
select count(*) AS CountOfShopping,
year(ModifiedDate) as FirstShopping
from SalesLT.Customer
group by year(ModifiedDate)
order by CountOfShopping desc
+-----------------+---------------+
| CountOfShopping | FirstShopping |
+-----------------+---------------+
|       313       |      2006     |
|       308       |      2005     |
|       210       |      2007     |
|        10       |      2009     |
|        6        |      2008     |
+-----------------+---------------+
Query 1 returned 5 records.



In [44]:
query = ["""
select count(*) as CountOfShopping,
YEAR(ModifiedDate) as FirstShopping
from SalesLT.Customer
group by YEAR(ModifiedDate)
having count(*) > 200
order by CountOfShopping desc"""]

fetch_and_display_queries(engine, query)

Executing query 1:

select count(*) as CountOfShopping,
YEAR(ModifiedDate) as FirstShopping
from SalesLT.Customer
group by YEAR(ModifiedDate)
having count(*) > 200
order by CountOfShopping desc
+-----------------+---------------+
| CountOfShopping | FirstShopping |
+-----------------+---------------+
|       313       |      2006     |
|       308       |      2005     |
|       210       |      2007     |
+-----------------+---------------+
Query 1 returned 3 records.



In [46]:
query = ["""select count(*) as CountOfCustomer 
from SalesLT.Customer
"""]

fetch_and_display_queries(engine, query)

Executing query 1:
select count(*) as CountOfCustomer 
from SalesLT.Customer

+-----------------+
| CountOfCustomer |
+-----------------+
|       847       |
+-----------------+
Query 1 returned 1 records.

